### N-gram language models or how to write scientific papers (4 pts)

We shall train our language model on a corpora of [ArXiv](http://arxiv.org/) articles and see if we can generate a new one!

![img](https://media.npr.org/assets/img/2013/12/10/istock-18586699-monkey-computer_brick-16e5064d3378a14e0e4c2da08857efe03c04695e-s800-c85.jpg)

_data by neelshah18 from [here](https://www.kaggle.com/neelshah18/arxivdataset/)_

_Disclaimer: this has nothing to do with actual science. But it's fun, so who cares?!_

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [74]:
# Alternative manual download link: https://yadi.sk/d/_nGyU2IajjR9-w
!wget "https://www.dropbox.com/s/99az9n1b57qkd9j/arxivData.json.tar.gz?dl=1" -O arxivData.json.tar.gz
!tar -xvzf arxivData.json.tar.gz
data = pd.read_json("./arxivData.json")
data.sample(n=5)

--2024-10-01 12:44:23--  https://www.dropbox.com/s/99az9n1b57qkd9j/arxivData.json.tar.gz?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.65.18, 2620:100:6017:18::a27d:212
Connecting to www.dropbox.com (www.dropbox.com)|162.125.65.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://www.dropbox.com/scl/fi/0mulrothty5o8i8ud9gz2/arxivData.json.tar.gz?rlkey=n759u5qx2xpxxglmrl390vwvk&dl=1 [following]
--2024-10-01 12:44:24--  https://www.dropbox.com/scl/fi/0mulrothty5o8i8ud9gz2/arxivData.json.tar.gz?rlkey=n759u5qx2xpxxglmrl390vwvk&dl=1
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc773f0b5ba3340361c920daa60d.dl.dropboxusercontent.com/cd/0/inline/CbmG063vpDygcyXfuo9adLyFDedzunYtSJCvdtHjHDWOyZsurxkx3jwJ18MveUabYFr-8md7touP3AWyj5-PDzQHleb4FThPiix63Ue61wKX1_JzbrffU2Q_Vf4iQdtSfMQ/file?dl=1# [following]
--2024-10-01 12:44:25--  https://uc773f0b5ba3340361c920daa60d.dl.dropboxu

,author,day,id,link,month,summary,tag,title,year
13200,"[{'name': 'Xiangrong Zeng'}, {'name': 'Mário A...",15,1409.4271v5,"[{'rel': 'alternate', 'href': 'http://arxiv.or...",9,The ordered weighted $\ell_1$ norm (OWL) was r...,"[{'term': 'cs.DS', 'scheme': 'http://arxiv.org...",The Ordered Weighted $\ell_1$ Norm: Atomic For...,2014
18478,"[{'name': 'Yongzhi Cao'}, {'name': 'Yoshinori ...",10,1012.2162v1,"[{'rel': 'alternate', 'href': 'http://arxiv.or...",12,Fuzzy automata have long been accepted as a ge...,"[{'term': 'cs.AI', 'scheme': 'http://arxiv.org...",Nondeterministic fuzzy automata,2010
33994,[{'name': 'Beryl Hoffman'}],20,cmp-lg/9407017v1,"[{'rel': 'alternate', 'href': 'http://arxiv.or...",7,Turkish has considerably freer word order than...,"[{'term': 'cmp-lg', 'scheme': 'http://arxiv.or...",Generating Context-Appropriate Word Orders in ...,1994
26267,"[{'name': 'Minkyoung Cho'}, {'name': 'Younggi ...",20,1609.06024v1,"[{'rel': 'alternate', 'href': 'http://arxiv.or...",9,The human activity recognition in the IoT envi...,"[{'term': 'cs.CV', 'scheme': 'http://arxiv.org...",Contextual Relationship-based Activity Segment...,2016
4985,"[{'name': 'Marc Pickett'}, {'name': 'Ayush Sek...",13,1707.03979v1,"[{'rel': 'alternate', 'href': 'http://arxiv.or...",7,Domain knowledge can often be encoded in the s...,"[{'term': 'cs.AI', 'scheme': 'http://arxiv.org...",A Brief Study of In-Domain Transfer and Learni...,2017


In [3]:
# assemble lines: concatenate title and description
lines = data.apply(lambda row: row['title'] + ' ; ' + row['summary'].replace("\n", ' '), axis=1).tolist()

sorted(lines, key=len)[:3]

['Differential Contrastive Divergence ; This paper has been retracted.',
 'What Does Artificial Life Tell Us About Death? ; Short philosophical essay',
 'P=NP ; We claim to resolve the P=?NP problem via a formal argument for P=NP.']

### Tokenization

You know the dril. The data is messy. Go clean the data. Use WordPunctTokenizer or something.


In [4]:
# Task: convert lines (in-place) into strings of space-separated tokens. Import & use WordPunctTokenizer
from nltk.tokenize import WordPunctTokenizer
tokenizer = WordPunctTokenizer()
print(lines[0])
lines = list(map(lambda x: ' '.join(tokenizer.tokenize(x)).lower() , lines))
print(lines[0])

Dual Recurrent Attention Units for Visual Question Answering ; We propose an architecture for VQA which utilizes recurrent layers to generate visual and textual attention. The memory characteristic of the proposed recurrent attention units offers a rich joint embedding of visual and textual features and enables the model to reason relations between several parts of the image and question. Our single model outperforms the first place winner on the VQA 1.0 dataset, performs within margin to the current state-of-the-art ensemble model. We also experiment with replacing attention mechanisms in other state-of-the-art models with our implementation and show increased accuracy. In both cases, our recurrent attention mechanism improves performance in tasks requiring sequential or relational reasoning on the VQA dataset.
dual recurrent attention units for visual question answering ; we propose an architecture for vqa which utilizes recurrent layers to generate visual and textual attention . the

In [5]:
assert sorted(lines, key=len)[0] == \
    'differential contrastive divergence ; this paper has been retracted .'
assert sorted(lines, key=len)[2] == \
    'p = np ; we claim to resolve the p =? np problem via a formal argument for p = np .'

### N-Gram Language Model (1point)

A language model is a probabilistic model that estimates text probability: the joint probability of all tokens $w_t$ in text $X$: $P(X) = P(w_1, \dots, w_T)$.

It can do so by following the chain rule:
$$P(w_1, \dots, w_T) = P(w_1)P(w_2 \mid w_1)\dots P(w_T \mid w_1, \dots, w_{T-1}).$$

The problem with such approach is that the final term $P(w_T \mid w_1, \dots, w_{T-1})$ depends on $n-1$ previous words. This probability is impractical to estimate for long texts, e.g. $T = 1000$.

One popular approximation is to assume that next word only depends on a finite amount of previous words:

$$P(w_t \mid w_1, \dots, w_{t - 1}) = P(w_t \mid w_{t - n + 1}, \dots, w_{t - 1})$$

Such model is called __n-gram language model__ where n is a parameter. For example, in 3-gram language model, each word only depends on 2 previous words.

$$
    P(w_1, \dots, w_n) = \prod_t P(w_t \mid w_{t - n + 1}, \dots, w_{t - 1}).
$$

You can also sometimes see such approximation under the name of _n-th order markov assumption_.

The first stage to building such a model is counting all word occurences given N-1 previous words

In [6]:
from tqdm import tqdm
from collections import defaultdict, Counter

# special tokens:
# - `UNK` represents absent tokens,
# - `EOS` is a special token after the end of sequence

UNK, EOS = "_UNK_", "_EOS_"

def count_ngrams(lines, n):
    """
    Count how many times each word occured after (n - 1) previous words
    :param lines: an iterable of strings with space-separated tokens
    :returns: a dictionary { tuple(prefix_tokens): {next_token_1: count_1, next_token_2: count_2}}

    When building counts, please consider the following two edge cases:
    - if prefix is shorter than (n - 1) tokens, it should be padded with UNK. For n=3,
      empty prefix: "" -> (UNK, UNK)
      short prefix: "the" -> (UNK, the)
      long prefix: "the new approach" -> (new, approach)
    - you should add a special token, EOS, at the end of each sequence
      "... with deep neural networks ." -> (..., with, deep, neural, networks, ., EOS)
      count the probability of this token just like all others.
    """
    counts = defaultdict(Counter)
    # counts[(word1, word2)][word3] = how many times word3 occured after (word1, word2)
    full_text = (str(' ' + EOS + ' ').join(lines) + ' ' + EOS).split()
    # print(full_text)

    prefix_list = [UNK]*(n-1)
    # print(prefix_list)
    prefix = tuple(prefix_list)
    for i in range(len(full_text)):
      word = full_text[i]
      # print('prefix', prefix, ' word ', word)
      # prefix = tuple(full_text[i:i+n])
      counts[prefix][word] +=1
      if word != EOS:
        prefix_list.append(word)
        prefix_list.pop(0)
        prefix = tuple(prefix_list)
      else:
        prefix_list = [UNK]*(n-1)
        prefix = tuple(prefix_list)
      # counts[prefix][word] +=1
      # prefix_list.append(word)

    return counts


In [26]:
dummy_lines = sorted(lines, key=len)[:2]
print(dummy_lines)
counts = count_ngrams(dummy_lines, n=1)
print(counts)

['differential contrastive divergence ; this paper has been retracted .', 'what does artificial life tell us about death ? ; short philosophical essay']
defaultdict(<class 'collections.Counter'>, {(): Counter({';': 2, '_EOS_': 2, 'differential': 1, 'contrastive': 1, 'divergence': 1, 'this': 1, 'paper': 1, 'has': 1, 'been': 1, 'retracted': 1, '.': 1, 'what': 1, 'does': 1, 'artificial': 1, 'life': 1, 'tell': 1, 'us': 1, 'about': 1, 'death': 1, '?': 1, 'short': 1, 'philosophical': 1, 'essay': 1})})


In [8]:
# let's test it
dummy_lines = sorted(lines, key=len)[:100]
dummy_counts = count_ngrams(dummy_lines, n=3)
assert set(map(len, dummy_counts.keys())) == {2}, "please only count {n-1}-grams"
assert len(dummy_counts[('_UNK_', '_UNK_')]) == 78
assert dummy_counts['_UNK_', 'a']['note'] == 3
assert dummy_counts['p', '=']['np'] == 2
assert dummy_counts['author', '.']['_EOS_'] == 1

Once we can count N-grams, we can build a probabilistic language model.
The simplest way to compute probabilities is in proporiton to counts:

$$ P(w_t | prefix) = { Count(prefix, w_t) \over \sum_{\hat w} Count(prefix, \hat w) } $$

In [69]:
class NGramLanguageModel:
    def __init__(self, lines, n):
        """
        Train a simple count-based language model:
        compute probabilities P(w_t | prefix) given ngram counts

        :param n: computes probability of next token given (n - 1) previous words
        :param lines: an iterable of strings with space-separated tokens
        """
        assert n >= 1
        self.n = n

        counts = count_ngrams(lines, self.n)
        # print(counts.keys())

        # compute token proabilities given counts
        self.probs = defaultdict(Counter)
        # probs[(word1, word2)][word3] = P(word3 | word1, word2)

        # populate self.probs with actual probabilities
        for prefix in counts.keys():
          words_given_prefix = dict(counts[prefix])
          # print('_________________')
          # print('prefix',prefix)
          # print(words_given_prefix)
          tot = np.sum(np.array(list(words_given_prefix.values())))
          # print(tot)
          for word, val in words_given_prefix.items():
            prob = val/tot
            self.probs[prefix][word] = prob

        # print(self.probs)


    def get_possible_next_tokens(self, prefix):
        """
        :param prefix: string with space-separated prefix tokens
        :returns: a dictionary {token : it's probability} for all tokens with positive probabilities
        """
        prefix = prefix.split()
        prefix = prefix[max(0, len(prefix) - self.n + 1):]
        prefix = [ UNK ] * (self.n - 1 - len(prefix)) + prefix
        return self.probs[tuple(prefix)]

    def get_next_token_prob(self, prefix, next_token):
        """
        :param prefix: string with space-separated prefix tokens
        :param next_token: the next token to predict probability for
        :returns: P(next_token|prefix) a single number, 0 <= P <= 1
        """
        return self.get_possible_next_tokens(prefix).get(next_token, 0)

In [38]:
print(dummy_lines)
dummy_lm = NGramLanguageModel(dummy_lines, n=1)

['differential contrastive divergence ; this paper has been retracted .', 'what does artificial life tell us about death ? ; short philosophical essay']
defaultdict(<class 'collections.Counter'>, {(): Counter({';': 0.08, '_EOS_': 0.08, 'differential': 0.04, 'contrastive': 0.04, 'divergence': 0.04, 'this': 0.04, 'paper': 0.04, 'has': 0.04, 'been': 0.04, 'retracted': 0.04, '.': 0.04, 'what': 0.04, 'does': 0.04, 'artificial': 0.04, 'life': 0.04, 'tell': 0.04, 'us': 0.04, 'about': 0.04, 'death': 0.04, '?': 0.04, 'short': 0.04, 'philosophical': 0.04, 'essay': 0.04})})


Let's test it!

In [11]:
dummy_lm = NGramLanguageModel(dummy_lines, n=3)

p_initial = dummy_lm.get_possible_next_tokens('') # '' -> ['_UNK_', '_UNK_']
assert np.allclose(p_initial['learning'], 0.02)
assert np.allclose(p_initial['a'], 0.13)
assert np.allclose(p_initial.get('meow', 0), 0)
assert np.allclose(sum(p_initial.values()), 1)

p_a = dummy_lm.get_possible_next_tokens('a') # '' -> ['_UNK_', 'a']
assert np.allclose(p_a['machine'], 0.15384615)
assert np.allclose(p_a['note'], 0.23076923)
assert np.allclose(p_a.get('the', 0), 0)
assert np.allclose(sum(p_a.values()), 1)

assert np.allclose(dummy_lm.get_possible_next_tokens('a note')['on'], 1)
assert dummy_lm.get_possible_next_tokens('a machine') == \
    dummy_lm.get_possible_next_tokens("there have always been ghosts in a machine"), \
    "your 3-gram model should only depend on 2 previous words"

Now that you've got a working n-gram language model, let's see what sequences it can generate. But first, let's train it on the whole dataset.

In [13]:
lm = NGramLanguageModel(lines, n=3)

The process of generating sequences is... well, it's sequential. You maintain a list of tokens and iteratively add next token by sampling with probabilities.

$ X = [] $

__forever:__
* $w_{next} \sim P(w_{next} | X)$
* $X = concat(X, w_{next})$


Instead of sampling with probabilities, one can also try always taking most likely token, sampling among top-K most likely tokens or sampling with temperature. In the latter case (temperature), one samples from

$$w_{next} \sim {P(w_{next} | X) ^ {1 / \tau} \over \sum_{\hat w} P(\hat w | X) ^ {1 / \tau}}$$

Where $\tau > 0$ is model temperature. If $\tau << 1$, more likely tokens will be sampled with even higher probability while less likely tokens will vanish.

In [14]:
def get_next_token(lm, prefix, temperature=1.0):
    """
    return next token after prefix;
    :param temperature: samples proportionally to lm probabilities ^ (1 / temperature)
        if temperature == 0, always takes most likely token. Break ties arbitrarily.
    """


    # print(lm.get_possible_next_tokens(prefix))
    tokens, probs = zip(*lm.get_possible_next_tokens(prefix).items())
    probs = np.array(probs)

    if temperature == 0:
      next_token = tokens[np.argmax(probs)]
      return next_token


    # print(tokens)

    inv_temp = (1/temperature)
    prob_sum_temp = np.sum(probs ** inv_temp)
    # print(prob_sum_temp)
    true_probs = (probs ** inv_temp) / prob_sum_temp
    # print(true_probs)

    next_token = np.random.choice(tokens, p=true_probs)
    return next_token


In [15]:
from collections import Counter
test_freqs = Counter([get_next_token(lm, 'there have') for _ in range(10000)])

print(test_freqs['not'], test_freqs['been'], test_freqs['lately'])
assert 250 < test_freqs['not'] < 450
assert 8500 < test_freqs['been'] < 9500
assert 1 < test_freqs['lately'] < 200

test_freqs = Counter([get_next_token(lm, 'deep', temperature=1.0) for _ in range(10000)])
assert 1500 < test_freqs['learning'] < 3000
test_freqs = Counter([get_next_token(lm, 'deep', temperature=0.5) for _ in range(10000)])
assert 8000 < test_freqs['learning'] < 9000
test_freqs = Counter([get_next_token(lm, 'deep', temperature=0.0) for _ in range(10000)])
assert test_freqs['learning'] == 10000

print("Looks nice!")

358 9160 71
Looks nice!


Let's have fun with this model

In [16]:
prefix = 'artificial' # <- your ideas :)

for i in range(100):
    prefix += ' ' + get_next_token(lm, prefix)
    if prefix.endswith(EOS) or len(lm.get_possible_next_tokens(prefix)) == 0:
        break

print(prefix)

artificial intelligence . as a local randomization scheme that transforms any input modality , it is more flexible for research . six widely - used embeddings pre - images to achieve transfer learning and recognition by combination of the archetypes , which includes the interaction between the given question was looking for sparse optimization which has been implemented for skew - symmetric tuning to cityscapes and camvid . _EOS_


In [17]:
prefix = 'bridging the' # <- more of your ideas

for i in range(100):
    prefix += ' ' + get_next_token(lm, prefix, temperature=0.5)
    if prefix.endswith(EOS) or len(lm.get_possible_next_tokens(prefix)) == 0:
        break

print(prefix)

bridging the gap between the two - stage stochastic optimization with multi - task learning ; the present work , we propose a novel approach for training a classifier to quantify the extent to which the paradigms of learning algorithms , for which we use a genetic algorithm based on a range of applications , e . g . activity - based approaches . _EOS_


__More in the homework:__ nucleus sampling, top-k sampling, beam search(not for the faint of heart).

### Evaluating language models: perplexity (1point)

Perplexity is a measure of how well your model approximates the true probability distribution behind the data. __Smaller perplexity = better model__.

To compute perplexity on one sentence, use:
$$
    {\mathbb{P}}(w_1 \dots w_N) = P(w_1, \dots, w_N)^{-\frac1N} = \left( \prod_t P(w_t \mid w_{t - n}, \dots, w_{t - 1})\right)^{-\frac1N},
$$


On the corpora level, perplexity is a product of probabilities of all tokens in all sentences to the power of $1/N$, where $N$ is __total length (in tokens) of all sentences__ in corpora.

This number can quickly get too small for float32/float64 precision, so we recommend you to first compute log-perplexity (from log-probabilities) and then take the exponent.

In [18]:
def perplexity(lm, lines, min_logprob=np.log(10 ** -50.)):
    """
    :param lines: a list of strings with space-separated tokens
    :param min_logprob: if log(P(w | ...)) is smaller than min_logprop, set it equal to min_logrob
    :returns: corpora-level perplexity - a single scalar number from the formula above

    Note: do not forget to compute P(w_first | empty) and P(eos | full_sequence)

    PLEASE USE lm.get_next_token_prob and NOT lm.get_possible_next_tokens
    """
    lines = str(' ' + EOS + ' ').join(lines) + ' ' + EOS
    print(lines)
    words = lines.split()
    log_perplexity = 0

    prefix_list = [UNK] * (lm.n - 1)
    prefix = ' '.join(prefix_list)

    for word in words:
      prob = lm.get_next_token_prob(prefix, word)
      log_prob = np.max([np.log(prob), min_logprob])
      # print(word, log_prob)
      log_perplexity += log_prob

      if word != EOS:
        prefix_list.append(word)
        prefix_list.pop(0)

      else:
        prefix_list = [UNK] * (lm.n - 1)

      prefix = ' '.join(prefix_list)

    log_perplexity *= -1/len(words)

    return np.exp(log_perplexity)

In [19]:
lm1 = NGramLanguageModel(dummy_lines, n=1)
lm3 = NGramLanguageModel(dummy_lines, n=3)
lm10 = NGramLanguageModel(dummy_lines, n=10)

ppx1 = perplexity(lm1, dummy_lines)
ppx3 = perplexity(lm3, dummy_lines)
ppx10 = perplexity(lm10, dummy_lines)
ppx_missing = perplexity(lm3, ['the jabberwock , with eyes of flame , '])  # thanks, L. Carrol

print("Perplexities: ppx1=%.3f ppx3=%.3f ppx10=%.3f" % (ppx1, ppx3, ppx10))

assert all(0 < ppx < 500 for ppx in (ppx1, ppx3, ppx10)), "perplexity should be non-negative and reasonably small"
assert ppx1 > ppx3 > ppx10, "higher N models should overfit and "
assert np.isfinite(ppx_missing) and ppx_missing > 10 ** 6, "missing words should have large but finite perplexity. " \
    " Make sure you use min_logprob right"
assert np.allclose([ppx1, ppx3, ppx10], (318.2132342216302, 1.5199996213739575, 1.1838145037901249))

differential contrastive divergence ; this paper has been retracted . _EOS_ what does artificial life tell us about death ? ; short philosophical essay _EOS_ p = np ; we claim to resolve the p =? np problem via a formal argument for p = np . _EOS_ computational geometry column 38 ; recent results on curve reconstruction are described . _EOS_ weak evolvability equals strong evolvability ; an updated version will be uploaded later . _EOS_ creating a new ontology : a modular approach ; creating a new ontology : a modular approach _EOS_ defeasible reasoning in oscar ; this is a system description for the oscar defeasible reasoner . _EOS_ essence ' description ; a description of the essence ' language as used by the tool savile row . _EOS_ deep neural networks - a brief history ; introduction to deep neural networks and their history . _EOS_ statistical physics for natural language processing ; this paper has been withdrawn by the author . _EOS_ complex networks ; introduction to the specia

<ipython-input-18-f153df83a6ba>:21: RuntimeWarning: divide by zero encountered in log
  log_prob = np.max([np.log(prob), min_logprob])


Now let's measure the actual perplexity: we'll split the data into train and test and score model on test data only.

In [20]:
from sklearn.model_selection import train_test_split
train_lines, test_lines = train_test_split(lines, test_size=0.25, random_state=42)

for n in (1, 2, 3):
    lm = NGramLanguageModel(n=n, lines=train_lines)
    ppx = perplexity(lm, test_lines)
    print("N = %i, Perplexity = %.5f" % (n, ppx))


Output hidden; open in https://colab.research.google.com to view.

In [ ]:
# whoops, it just blew up :)

### LM Smoothing

The problem with our simple language model is that whenever it encounters an n-gram it has never seen before, it assigns it with the probabilitiy of 0. Every time this happens, perplexity explodes.

To battle this issue, there's a technique called __smoothing__. The core idea is to modify counts in a way that prevents probabilities from getting too low. The simplest algorithm here is Additive smoothing (aka [Lapace smoothing](https://en.wikipedia.org/wiki/Additive_smoothing)):

$$ P(w_t | prefix) = { Count(prefix, w_t) + \delta \over \sum_{\hat w} (Count(prefix, \hat w) + \delta) } $$

If counts for a given prefix are low, additive smoothing will adjust probabilities to a more uniform distribution. Not that the summation in the denominator goes over _all words in the vocabulary_.

Here's an example code we've implemented for you:

In [21]:
class LaplaceLanguageModel(NGramLanguageModel):
    """ this code is an example, no need to change anything """
    def __init__(self, lines, n, delta=1.0):
        self.n = n
        counts = count_ngrams(lines, self.n)
        self.vocab = set(token for token_counts in counts.values() for token in token_counts)
        self.probs = defaultdict(Counter)

        for prefix in counts:
            token_counts = counts[prefix]
            total_count = sum(token_counts.values()) + delta * len(self.vocab)
            self.probs[prefix] = {token: (token_counts[token] + delta) / total_count
                                          for token in token_counts}
    def get_possible_next_tokens(self, prefix):
        token_probs = super().get_possible_next_tokens(prefix)
        missing_prob_total = 1.0 - sum(token_probs.values())
        missing_prob = missing_prob_total / max(1, len(self.vocab) - len(token_probs))
        return {token: token_probs.get(token, missing_prob) for token in self.vocab}

    def get_next_token_prob(self, prefix, next_token):
        token_probs = super().get_possible_next_tokens(prefix)
        if next_token in token_probs:
            return token_probs[next_token]
        else:
            missing_prob_total = 1.0 - sum(token_probs.values())
            missing_prob_total = max(0, missing_prob_total) # prevent rounding errors
            return missing_prob_total / max(1, len(self.vocab) - len(token_probs))


**Disclaimer**: the implementation above assumes all words unknown within a given context to be equally likely, *as well as the words outside of vocabulary*. Therefore, its' perplexity will be lower than it should when encountering such words. Therefore, comparing it with a model with fewer unknown words will not be fair. When implementing your own smoothing, you may handle this by adding a virtual `UNK` token of non-zero probability. Technically, this will result in a model where probabilities do not add up to $1$, but it is close enough for a practice excercise.

In [22]:
#test that it's a valid probability model
for n in (1, 2, 3):
    dummy_lm = LaplaceLanguageModel(dummy_lines, n=n)
    assert np.allclose(sum([dummy_lm.get_next_token_prob('a', w_i) for w_i in dummy_lm.vocab]), 1), "I told you not to break anything! :)"

In [23]:
for n in (1, 2, 3):
    lm = LaplaceLanguageModel(train_lines, n=n, delta=0.1)
    ppx = perplexity(lm, test_lines)
    print("N = %i, Perplexity = %.5f" % (n, ppx))

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
# optional: try to sample tokens from such a model

### Kneser-Ney smoothing (2 points)

Additive smoothing is simple, reasonably good but definitely not a State of The Art algorithm.


Your final task in this notebook is to implement [Kneser-Ney](https://en.wikipedia.org/wiki/Kneser%E2%80%93Ney_smoothing) smoothing.

It can be computed recurrently, for n>1:

$$P_{kn}(w_t | prefix_{n-1}) = { \max(0, Count(prefix_{n-1}, w_t) - \delta) \over \sum_{\hat w} Count(prefix_{n-1}, \hat w)} + \lambda_{prefix_{n-1}} \cdot P_{kn}(w_t | prefix_{n-2})$$

where
- $prefix_{n-1}$ is a tuple of {n-1} previous tokens
- $lambda_{prefix_{n-1}}$ is a normalization constant chosen so that probabilities add up to 1
- Unigram $P_{kn}(w_t | prefix_{n-2})$ corresponds to Kneser Ney smoothing for {N-1}-gram language model.
- Unigram $P_{kn}(w_t)$ is a special case: how likely it is to see x_t in an unfamiliar context

See lecture slides or wiki for more detailed formulae.

__Your task__ is to
- implement `KneserNeyLanguageModel` class,
- test it on 1-3 gram language models
- find optimal (within reason) smoothing delta for 3-gram language model with Kneser-Ney smoothing

In [70]:
class KneserNeyLanguageModel(NGramLanguageModel):
    """ A template for Kneser-Ney language model. Default delta may be suboptimal. """
    def __init__(self, lines, n, delta=1.0):
        self.n = n
        self.counts = count_ngrams(lines, self.n)
        self.vocab = set((str(' ' + EOS + ' ').join(lines) + ' ' + EOS).split())

        self.probs = self.calculate_probs(lines, delta)
        # print(f'for n={self.n}',self.probs)


    def calculate_probs(self, lines, delta):
        probs = defaultdict(Counter)

        if self.n == 1:
            words1 = np.array(list(self.counts[()].values()))

            probs[()] = dict(zip(self.counts[()].keys(), words1 / sum(words1)))
            return probs

        model_n_1 = KneserNeyLanguageModel(lines=lines, n=self.n-1, delta=delta)
        probs_n_1 = model_n_1.probs

        # print(f'probs for n-1 = {model_n_1.n}', probs_n_1)

        for prefix in self.counts.keys():
            words_given_prefix = self.counts[prefix]

            total_counts = sum(words_given_prefix.values())
            normalization = (delta/total_counts) * len(words_given_prefix)

            # print('_________________')
            # print('prefix',prefix)
            # print('words given prefix', words_given_prefix)
            # print('total counts given prefix',total_counts)
            # print('normalization', normalization)

            for word, val in words_given_prefix.items():
                prob = np.max([val - delta, 0])/total_counts + normalization * probs_n_1[prefix[:-1]][word]
                probs[prefix][word] = prob

        return probs



    def get_possible_next_tokens(self, prefix):
        token_probs = super().get_possible_next_tokens(prefix)
        missing_prob_total = 1.0 - sum(token_probs.values())
        missing_prob = missing_prob_total / max(1, len(self.vocab) - len(token_probs))
        return {token: token_probs.get(token, missing_prob) for token in self.vocab}


    def get_next_token_prob(self, prefix, next_token):
        token_probs = super().get_possible_next_tokens(prefix)
        if next_token in token_probs:
            return token_probs[next_token]
        else:
            missing_prob_total = 1.0 - sum(token_probs.values())
            missing_prob_total = max(0, missing_prob_total) # prevent rounding errors
            return missing_prob_total / max(1, len(self.vocab) - len(token_probs))


In [71]:
print(dummy_lines)
dummy_lm = KneserNeyLanguageModel(dummy_lines, n=n)

print([dummy_lm.get_next_token_prob('a', w_i) for w_i in dummy_lm.vocab])
print(sum([dummy_lm.get_next_token_prob('a', w_i) for w_i in dummy_lm.vocab]))

['differential contrastive divergence ; this paper has been retracted .', 'what does artificial life tell us about death ? ; short philosophical essay']
[0.043478260869565216, 0.043478260869565216, 0.043478260869565216, 0.043478260869565216, 0.043478260869565216, 0.043478260869565216, 0.043478260869565216, 0.043478260869565216, 0.043478260869565216, 0.043478260869565216, 0.043478260869565216, 0.043478260869565216, 0.043478260869565216, 0.043478260869565216, 0.043478260869565216, 0.043478260869565216, 0.043478260869565216, 0.043478260869565216, 0.043478260869565216, 0.043478260869565216, 0.043478260869565216, 0.043478260869565216, 0.043478260869565216]
0.9999999999999996


In [72]:
#test that it's a valid probability model
print(dummy_lines)
for n in (1, 2, 3):
    dummy_lm = KneserNeyLanguageModel(dummy_lines, n=n)
    assert np.allclose(sum([dummy_lm.get_next_token_prob('a', w_i) for w_i in dummy_lm.vocab]), 1), "I told you not to break anything! :)"

['differential contrastive divergence ; this paper has been retracted .', 'what does artificial life tell us about death ? ; short philosophical essay']


In [73]:
for n in (1, 2, 3):
    lm = KneserNeyLanguageModel(train_lines, n=n, delta=1)
    ppx = perplexity(lm, test_lines)
    print("N = %i, Perplexity = %.5f" % (n, ppx))

Output hidden; open in https://colab.research.google.com to view.